In [1]:
import sympy as sp

In [2]:
E, A, I, phi, L = sp.symbols('E A I phi L')
a, b, pi, pj = sp.symbols('a b p_i p_j')

### DEFINICIÓN DE MATRIZ DE RIGIDEZ LOCAL Y VECTOR DE CARGAS LOCAL

In [3]:
k11 = E * A / L
k22 = 12 * E * I / (L**3 * (1 + phi))
k23 = 6 * E * I / (L**2 * (1 + phi))
k33 = (4 + phi) * E * I / (L * (1 + phi))
k66 = (2 - phi) * E * I / (L * (1 + phi))

k = sp.Matrix([
    [ k11,   0,    0,   -k11,   0,    0   ],
    [  0,   k22,  k23,    0,   -k22,  k23 ],
    [  0,   k23,  k33,    0,   -k23,  k66 ],
    [-k11,   0,    0,    k11,   0,    0   ],
    [  0,  -k22, -k23,    0,    k22, -k23 ],
    [  0,   k23,  k66,    0,   -k23,  k33 ]
])
k

Matrix([
[ A*E/L,                        0,                           0, -A*E/L,                        0,                           0],
[     0,  12*E*I/(L**3*(phi + 1)),      6*E*I/(L**2*(phi + 1)),      0, -12*E*I/(L**3*(phi + 1)),      6*E*I/(L**2*(phi + 1))],
[     0,   6*E*I/(L**2*(phi + 1)), E*I*(phi + 4)/(L*(phi + 1)),      0,  -6*E*I/(L**2*(phi + 1)), E*I*(2 - phi)/(L*(phi + 1))],
[-A*E/L,                        0,                           0,  A*E/L,                        0,                           0],
[     0, -12*E*I/(L**3*(phi + 1)),     -6*E*I/(L**2*(phi + 1)),      0,  12*E*I/(L**3*(phi + 1)),     -6*E*I/(L**2*(phi + 1))],
[     0,   6*E*I/(L**2*(phi + 1)), E*I*(2 - phi)/(L*(phi + 1)),      0,  -6*E*I/(L**2*(phi + 1)), E*I*(phi + 4)/(L*(phi + 1))]])

In [4]:
N1 = (2 * pi + pj) * L / 6
V1 = b * L / 2 + a * L**2 / 60 * ((10 * phi + 9) / (phi + 1))
M1 = b * L**2 / 12 + a * L**3 / 120 * ((5 * phi + 4) / (phi + 1))
N2 = (pi + 2 * pj) * L / 6
V2 = b * L / 2 + a * L**2 / 60 * ((20 * phi + 21) / (phi + 1))
M2 = -(b * L**2 / 12 + a * L**3 / 120 * ((5 * phi + 6) / (phi + 1)))
q = sp.Matrix([N1, V1, M1, N2, V2, M2])
q

Matrix([
[                              L*(2*p_i + p_j)/6],
[     L**2*a*(10*phi + 9)/(60*(phi + 1)) + L*b/2],
[ L**3*a*(5*phi + 4)/(120*(phi + 1)) + L**2*b/12],
[                              L*(p_i + 2*p_j)/6],
[    L**2*a*(20*phi + 21)/(60*(phi + 1)) + L*b/2],
[-L**3*a*(5*phi + 6)/(120*(phi + 1)) - L**2*b/12]])

## **LIBERACION DE ROTACIONES**

### **rotacion inicial**

In [5]:
freeDof = [2]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 2
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [6]:
kp

Matrix([
[ A*E/L,                        0, 0, -A*E/L,                        0,                        0],
[     0,  12*E*I/(L**3*(phi + 4)), 0,      0, -12*E*I/(L**3*(phi + 4)),  12*E*I/(L**2*(phi + 4))],
[     0,                        0, 0,      0,                        0,                        0],
[-A*E/L,                        0, 0,  A*E/L,                        0,                        0],
[     0, -12*E*I/(L**3*(phi + 4)), 0,      0,  12*E*I/(L**3*(phi + 4)), -12*E*I/(L**2*(phi + 4))],
[     0,  12*E*I/(L**2*(phi + 4)), 0,      0, -12*E*I/(L**2*(phi + 4)),     12*E*I/(L*(phi + 4))]])

In [7]:
qp

Matrix([
[                                       L*(2*p_i + p_j)/6],
[ L*(5*L*a*phi + 12*L*a + 15*b*phi + 45*b)/(30*(phi + 4))],
[                                                       0],
[                                       L*(p_i + 2*p_j)/6],
[L*(10*L*a*phi + 48*L*a + 15*b*phi + 75*b)/(30*(phi + 4))],
[                     L**2*(-8*L*a - 15*b)/(30*(phi + 4))]])

### **rotacion final**

In [8]:
freeDof = [5]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 5
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [9]:
kp

Matrix([
[ A*E/L,                        0,                        0, -A*E/L,                        0, 0],
[     0,  12*E*I/(L**3*(phi + 4)),  12*E*I/(L**2*(phi + 4)),      0, -12*E*I/(L**3*(phi + 4)), 0],
[     0,  12*E*I/(L**2*(phi + 4)),     12*E*I/(L*(phi + 4)),      0, -12*E*I/(L**2*(phi + 4)), 0],
[-A*E/L,                        0,                        0,  A*E/L,                        0, 0],
[     0, -12*E*I/(L**3*(phi + 4)), -12*E*I/(L**2*(phi + 4)),      0,  12*E*I/(L**3*(phi + 4)), 0],
[     0,                        0,                        0,      0,                        0, 0]])

In [10]:
qp

Matrix([
[                                       L*(2*p_i + p_j)/6],
[ L*(5*L*a*phi + 27*L*a + 15*b*phi + 75*b)/(30*(phi + 4))],
[                      L**2*(7*L*a + 15*b)/(30*(phi + 4))],
[                                       L*(p_i + 2*p_j)/6],
[L*(10*L*a*phi + 33*L*a + 15*b*phi + 45*b)/(30*(phi + 4))],
[                                                       0]])

### **ambos rotaciones**

In [11]:

dof = 2
vecudof = sp.zeros(6, 1)
if kp[dof, dof] != 0.0:
    vecudof[dof] = qp[dof] / kp[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = qp - kp @ vecudof
qp = sp.simplify(qp)


freeDof = [2, 5]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)


In [12]:
kp

Matrix([
[ A*E/L, 0, 0, -A*E/L, 0, 0],
[     0, 0, 0,      0, 0, 0],
[     0, 0, 0,      0, 0, 0],
[-A*E/L, 0, 0,  A*E/L, 0, 0],
[     0, 0, 0,      0, 0, 0],
[     0, 0, 0,      0, 0, 0]])

In [13]:
qp

Matrix([
[L*(2*p_i + p_j)/6],
[  L*(L*a + 3*b)/6],
[                0],
[L*(p_i + 2*p_j)/6],
[L*(2*L*a + 3*b)/6],
[                0]])

## **Liberacion de axial (solo de pude uno de ellos)**

### **axial inicial**

In [14]:
freeDof = [0]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 0
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [15]:
kp

Matrix([
[0,                        0,                           0, 0,                        0,                           0],
[0,  12*E*I/(L**3*(phi + 1)),      6*E*I/(L**2*(phi + 1)), 0, -12*E*I/(L**3*(phi + 1)),      6*E*I/(L**2*(phi + 1))],
[0,   6*E*I/(L**2*(phi + 1)), E*I*(phi + 4)/(L*(phi + 1)), 0,  -6*E*I/(L**2*(phi + 1)), E*I*(2 - phi)/(L*(phi + 1))],
[0,                        0,                           0, 0,                        0,                           0],
[0, -12*E*I/(L**3*(phi + 1)),     -6*E*I/(L**2*(phi + 1)), 0,  12*E*I/(L**3*(phi + 1)),     -6*E*I/(L**2*(phi + 1))],
[0,   6*E*I/(L**2*(phi + 1)), E*I*(2 - phi)/(L*(phi + 1)), 0,  -6*E*I/(L**2*(phi + 1)), E*I*(phi + 4)/(L*(phi + 1))]])

In [16]:
qp

Matrix([
[                                                       0],
[    L*(L*a*(10*phi + 9) + 30*b*(phi + 1))/(60*(phi + 1))],
[ L**2*(L*a*(5*phi + 4) + 10*b*(phi + 1))/(120*(phi + 1))],
[                                         L*(p_i + p_j)/2],
[   L*(L*a*(20*phi + 21) + 30*b*(phi + 1))/(60*(phi + 1))],
[L**2*(-L*a*(5*phi + 6) - 10*b*(phi + 1))/(120*(phi + 1))]])

### **Axial final**

In [17]:
freeDof = [3]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 3
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [18]:
kp

Matrix([
[0,                        0,                           0, 0,                        0,                           0],
[0,  12*E*I/(L**3*(phi + 1)),      6*E*I/(L**2*(phi + 1)), 0, -12*E*I/(L**3*(phi + 1)),      6*E*I/(L**2*(phi + 1))],
[0,   6*E*I/(L**2*(phi + 1)), E*I*(phi + 4)/(L*(phi + 1)), 0,  -6*E*I/(L**2*(phi + 1)), E*I*(2 - phi)/(L*(phi + 1))],
[0,                        0,                           0, 0,                        0,                           0],
[0, -12*E*I/(L**3*(phi + 1)),     -6*E*I/(L**2*(phi + 1)), 0,  12*E*I/(L**3*(phi + 1)),     -6*E*I/(L**2*(phi + 1))],
[0,   6*E*I/(L**2*(phi + 1)), E*I*(2 - phi)/(L*(phi + 1)), 0,  -6*E*I/(L**2*(phi + 1)), E*I*(phi + 4)/(L*(phi + 1))]])

In [19]:
qp

Matrix([
[                                         L*(p_i + p_j)/2],
[    L*(L*a*(10*phi + 9) + 30*b*(phi + 1))/(60*(phi + 1))],
[ L**2*(L*a*(5*phi + 4) + 10*b*(phi + 1))/(120*(phi + 1))],
[                                                       0],
[   L*(L*a*(20*phi + 21) + 30*b*(phi + 1))/(60*(phi + 1))],
[L**2*(-L*a*(5*phi + 6) - 10*b*(phi + 1))/(120*(phi + 1))]])

## **LIBERACION DE CORTANTES**

### **cortante inicial**

In [21]:
freeDof = [1]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 1
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [22]:
kp

Matrix([
[ A*E/L, 0,      0, -A*E/L, 0,      0],
[     0, 0,      0,      0, 0,      0],
[     0, 0,  E*I/L,      0, 0, -E*I/L],
[-A*E/L, 0,      0,  A*E/L, 0,      0],
[     0, 0,      0,      0, 0,      0],
[     0, 0, -E*I/L,      0, 0,  E*I/L]])

In [23]:
qp

Matrix([
[   L*(2*p_i + p_j)/6],
[                   0],
[L**2*(-L*a - 4*b)/24],
[   L*(p_i + 2*p_j)/6],
[     L*(L*a + 2*b)/2],
[ L**2*(-L*a/8 - b/3)]])

### **cortante final**

In [24]:
freeDof = [4]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 4
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [25]:
kp

Matrix([
[ A*E/L, 0,      0, -A*E/L, 0,      0],
[     0, 0,      0,      0, 0,      0],
[     0, 0,  E*I/L,      0, 0, -E*I/L],
[-A*E/L, 0,      0,  A*E/L, 0,      0],
[     0, 0,      0,      0, 0,      0],
[     0, 0, -E*I/L,      0, 0,  E*I/L]])

In [26]:
qp

Matrix([
[    L*(2*p_i + p_j)/6],
[      L*(L*a + 2*b)/2],
[L**2*(5*L*a + 8*b)/24],
[    L*(p_i + 2*p_j)/6],
[                    0],
[   L**2*(L*a/8 + b/6)]])

### **cortante ambos**

## **LIBERACIONES COMBINADAS**

In [ ]:
freeDof = [0, 1, 5]

k1 = k[:,freeDof]
k2 = k[freeDof,:]
k3 = sp.Matrix([k[freeDof,freeDof]])
invk3 = sp.Inverse(k3)
kp = k - k1 @ invk3 @ k2
kp = sp.simplify(kp)

dof = 4
vecudof = sp.zeros(6, 1)
if k[dof, dof] != 0.0:
    vecudof[dof] = q[dof] / k[dof, dof]
else:
    raise ZeroDivisionError(f"El término k[{dof},{dof}] es cero, no se puede aplicar release.")

# Nuevo vector de cargas con liberación
qp = q - k @ vecudof
qp = sp.simplify(qp)

In [33]:
kp

Matrix([
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0]])